In [1]:
data_var = '2023-12-14'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8491,2023-12-14,Uruguai Liga Uruguaia,21:00,Aguada,Defensor Sporting,1.56,2.32,158.5,1.86,1.84,-4.5,2.01,1.72,d6nKz63d,0.641026,0.431034,0.537634,0.543478,0.072060,137.638,23.850661,0.173285,2.4,1.341641,0.559017,169.20,1.514,0.246536,0.162838,57.0,132.042,27.606211,0.209071,1.8,1.643168,0.912871,150.88,1.592,0.304664,0.191372,18.0,94,92,1.80,1.64,0.000,0.000,0.277011,0.007644,0.109952,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
8492,2023-12-14,Macedônia Do Norte Prva Liga,14:00,TFT Skopje,Rabotnicki,2.97,1.36,150.5,1.84,1.86,4.5,1.85,1.85,Ob7ZBTBJ,0.336700,0.735294,0.543478,0.537634,0.071994,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,79,86,2.37,1.23,0.000,0.000,0.525839,0.007644,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
8493,2023-12-14,Rússia Liga Vtb United,14:00,Lokomotiv Kuban,Enisey,1.33,3.38,164.5,1.82,1.94,-8.5,2.00,1.77,4doSU5Pd,0.751880,0.295858,0.549451,0.515464,0.047738,120.248,25.797297,0.214534,2.4,1.341641,0.559017,116.80,1.462,0.322521,0.220603,39.0,158.188,68.243988,0.431411,1.8,1.643168,0.912871,112.14,1.886,1.016774,0.539117,32.0,80,89,1.46,1.26,0.000,208.400,0.615528,0.045134,0.086278,0.00,0.000,inf,0.000000,0.0,0.000000,1.16,0.232,10.258621,0.0,0.0,0.0
8494,2023-12-14,Reino Unido Bbl,16:45,Manchester Giants,Bristol Flyers,1.93,1.79,168.5,1.81,1.85,-6.5,1.98,1.74,lxZi2AJE,0.518135,0.558659,0.552486,0.540541,0.076794,304.006,216.266132,0.711388,1.2,1.643168,1.369306,282.40,3.664,2.608607,0.711956,-26.0,174.106,53.699413,0.308429,1.2,1.643168,1.369306,227.84,2.154,0.568005,0.263698,-2.0,80,89,3.53,2.56,0.000,169.216,0.053223,0.015456,0.091240,0.00,0.000,inf,0.000000,0.0,0.000000,-2.44,-0.488,-1.618852,0.0,0.0,0.0
8495,2023-12-14,Polônia Liga De Basquete,13:30,Trefl Sopot,GTK Gliwice,1.21,4.26,171.5,1.86,1.88,-11.5,2.05,1.69,0foXgIQn,0.826446,0.234742,0.537634,0.531915,0.061188,130.674,18.927395,0.144844,3.0,0.000000,0.000000,143.00,1.466,0.300050,0.204673,63.0,256.002,83.603050,0.326572,1.8,1.643168,0.912871,194.22,2.928,0.860389,0.293849,7.0,110,83,1.30,2.34,0.000,295.752,0.788547,0.007563,0.136128,0.00,0.000,inf,0.000000,0.0,0.000000,1.72,0.344,9.476744,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8567,2023-12-14,Chipre Divisão A,14:30,Anorthosis,Apop Paphou,1.01,9.70,154.5,1.85,1.85,-19.5,2.08,1.66,4d9uxMel,0.990099,0.103093,0.540541,0.540541,0.093192,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,342.026,272.700366,0.797309,0.6,1.341641,2.236068,567.80,5.052,3.736612,0.739630,-56.0,72,68,5.35,8.35,0.000,0.000,1.147480,0.000000,0.158815,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
8568,2023-12-14,China Wcba Feminina,08:30,Shandong F,Beijing F,1.49,2.50,152.5,1.82,1.88,-7.5,2.07,1.67,MXRZPWcA,0.671141,0.400000,0.549451,0.531915,0.071141,166.664,170.520779,1.023141,1.8,1.643168,0.912871,90.78,2.172,2.261099,1.041022,20.0,94.730,22.818549,0.240880,1.8,1.643168,0.912871,91.02,

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
8494,16:45,Reino Unido Bbl,Manchester Giants,Bristol Flyers,1.93,Back Home
8510,21:10,Mundo Liga Dos Campeões Das Américas,Hebraica y Macabi,Boca Juniors,2.07,Back Home
8513,12:00,Rússia Superliga,Zenit Petersburg 2,Barnaul,2.02,Back Home
8517,15:30,Países Baixos Copa Dbl,Groningen,Zwolle,1.66,Back Home
8518,09:30,Irã Superliga,Palayesh Naft Abadan,Mahram Tehran,2.20,Back Home
8520,16:15,Islândia Premier League,Hottur,Alftanes,2.32,Back Home
8525,13:30,Finlândia I Divisão A,Huima,Raholan Pyrkiva,1.84,Back Home
8532,15:30,Bósnia E Herzegovina Prvenstvo Bih,KK Bosna,Siroki Brijeg,1.99,Back Home
8534,05:00,Austrália Liga Feminina Wnbl,Southside F,Townsville F,2.03,Back Home
8540,16:30,Europa Euroliga,Baskonia,Virtus Bologna,1.67,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
8510,21:10,Mundo Liga Dos Campeões Das Américas,Hebraica y Macabi,Boca Juniors,1.85,Back Home
8534,05:00,Austrália Liga Feminina Wnbl,Southside F,Townsville F,1.87,Back Home
8561,15:15,Dinamarca Liga De Basquete,Randers,Horsens,1.86,Back Home


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
8538,17:00,Europa Euroliga,Lyon-Villeurbanne,Panathinaikos,2.74,Back Home
8552,08:35,China Cba,Shanxi Zhongyu,Shenzhen,2.32,Back Home
